In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import time
import matplotlib.pyplot as plt
from math import sqrt

In [2]:
spatient = 'patient6'

In [3]:
patient = pd.read_csv(spatient + '.csv')

In [4]:
display(patient.dtypes)

id                     object
timestamp_rounded      object
LIS1_x                  int64
LIS1_y                  int64
LIS1_z                  int64
LIS2_x                  int64
LIS2_y                  int64
LIS2_z                  int64
LSM_x                 float64
LSM_y                 float64
LSM_z                 float64
BMP                     int64
butterworth_filter      int64
timestamp              object
label                 float64
id_patient              int64
id_label               object
dtype: object

In [5]:
patient['id_500'] = (patient.index / 500).astype(int)
patient['LIS1'] = (patient['LIS1_x']*patient['LIS1_x'] + patient['LIS1_y']*patient['LIS1_y'] + patient['LIS1_z']*patient['LIS1_z'])
patient['LIS2'] = (patient['LIS2_x']*patient['LIS2_x'] + patient['LIS2_y']*patient['LIS2_y'] + patient['LIS2_z']*patient['LIS2_z'])
patient['wpr'] = patient['LSM_y']*patient['LSM_y'] + patient['LSM_z']*patient['LSM_z']
patient['wpry'] = patient['LSM_x']*patient['LSM_x']  + patient['LSM_y']*patient['LSM_y'] + patient['LSM_z']*patient['LSM_z']

In [6]:
def agrupa(x):    
    d = {}
    
    d['timestamp_rounded'] = x['timestamp_rounded'].iloc[0]
    
    d['LSM_x_std'] = x['LSM_x'].std()
    d['LSM_x_mean'] = np.absolute(x['LSM_x'].mean())
    d['LSM_y_std'] = x['LSM_y'].std()
    d['LSM_y_mean'] = np.absolute(x['LSM_y'].mean())
    d['LSM_z_std'] = x['LSM_z'].std()
    d['LSM_z_mean'] = np.absolute(x['LSM_z'].mean())
    d['LIS1_std'] = x['LIS1'].std()
    d['LIS2_std'] = x['LIS2'].std()
    d['LIS1'] = x['LIS1'].mean() / 1e6
    d['LIS2'] = x['LIS2'].mean() / 1e6
    d['wpr_mean'] = x['wpr'].mean()
    d['wpr_std'] = x['wpr'].std()
    d['wpry_mean'] = x['wpry'].mean()
    d['wpry_std'] = x['wpry'].std()
    d['BMP'] = x['BMP'].mean()
    s = x['butterworth_filter'].sum()
    sz = x['butterworth_filter'].size
    d['bw0'] = (sz-s)/sz
    d['bw1'] = s/sz
    d['timestamp'] = x['timestamp'].iloc[0]
    d['id_patient'] = x['id_patient'].iloc[0]
    d['label'] = x['label'].iloc[0]
    d['id_label'] = x['id_label'].iloc[0]
    return pd.Series(d)

In [7]:
df_agrupat = patient.groupby('id_500').apply(agrupa)

In [8]:
def trem_giro():
    df = df_agrupat['LSM_x_std'] > 1.7 * df_agrupat['LSM_x_mean']
    df = df & (df_agrupat['LSM_y_std'] > 1.5 * df_agrupat['LSM_y_mean'])
    df = df & (df_agrupat['LSM_z_std'] > 1.5 * df_agrupat['LSM_z_mean'])
    return df

In [9]:
df_agrupat['trem_giro'] = (trem_giro()).astype(int)
df_agrupat = df_agrupat.drop(columns=['LSM_x_std', 'LSM_y_std', 'LSM_z_std', 'LSM_x_mean', 'LSM_y_mean', 'LSM_z_mean'])

In [10]:
df_agrupat.to_csv(spatient + 'reduit.csv')